# Azure OpenAI Service - Chat on private data using LangChain

Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values:

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

Then, let's install all dependencies:

In [1]:
! pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached azure_identity-1.6.0-py2.py3-none-any.whl (108 kB)
  Using cached langchain-0.0.129-py3-none-any.whl (467 kB)


ERROR: Could not find a version that satisfies the requirement faiss==1.5.3 (from versions: none)
ERROR: No matching distribution found for faiss==1.5.3


In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()
# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0, openai_api_version="2023-05-15")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

First, we load up our documents from the `data` directory:

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter

loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [5]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

Now let's create a chain that can do the whole chat on our embedding database:

In [6]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

Now let's ask a question:

In [7]:
chat_history = []
query = "what is azure openai service?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

Azure OpenAI is a cloud-based service by Microsoft that allows users to access OpenAI's language models, including GPT-3, Codex, and Embeddings model series. These models can be used for various natural language processing tasks such as content generation, summarization, semantic search, and natural language to code translation. The service can be accessed through REST APIs, Python SDK or the web-based interface in the Azure OpenAI Studio. The service also includes Responsible AI features, including content filtering and a use case review process to ensure the safe and ethical use of the models.


We can use this to easy implement chat conversations:

In [8]:
chat_history = []

query = "what is Azure OpenAI Service?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

chat_history = [(query, result["answer"])]
query = "Which regions does the service support?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

Question: what is Azure OpenAI Service?
Answer: Azure OpenAI is a service that provides access to OpenAI's advanced language models, such as GPT-3, Codex, and Embeddings, for various natural language processing tasks. The service can be used through REST APIs, Python SDK, or the Azure OpenAI Studio web-based interface. The service comes with several features, such as fine-tuning, virtual network support, responsible AI, and content filtering. To get access to the Azure OpenAI service, users must apply for access using a form available on the Azure website.
Question: Which regions does the service support?
Answer: The Azure OpenAI service is currently available in East US, South Central US, and West Europe.
